# 1.1 Validação de Entrada e Saída

Este notebook compara os dados do mês atual com o mês anterior para identificar:
- Novos beneficiários (Entraram)
- Beneficiários cancelados (Saíram)

**Entradas:**
- `dados_atualizados.xlsx` (Mês Atual)
- Planilha do Mês Anterior (Configurável)

**Saídas (Opcional):**
- `relatorio_entradas.xlsx`
- `relatorio_saidas.xlsx`

In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

### Carregamento dos Dados
Carregue a planilha atual e a planilha anterior para comparação.

In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
dados_atual = pd.read_excel('../Data/dados.xlsx', skiprows=1, index_col=0, sheet_name="OUT 2025")

# Carrega a planilha do MÊS ANTERIOR
# Ajuste o caminho e a aba conforme necessário
dados_anterior = pd.read_excel('../Data/dados.xlsx', skiprows=1, index_col=0, sheet_name="SET 2025")
dados_anterior = dados_anterior.iloc[:-25] 

# Nomes para exibição no relatório
nome_mes_anterior = 'SETEMBRO'
nome_mes_atual = 'OUTUBRO'

In [ ]:
print("Visualizando Mês Atual:")
display(dados_atual.head())
print("\nVisualizando Mês Anterior:")
display(dados_anterior.head())

### Função de Comparação
Lógica principal para cruzar os dados.

In [ ]:
def compare_months(df1, df2, month1_name, month2_name, gerar_df_entrada=False, gerar_df_saida=False):
    """
    Compara dois dataframes (meses) e identifica nomes presentes em um mas não no outro,
    ou presentes em ambos com diferença na coluna 'MENSALIDADE' (NaN vs não NaN).
    """
    # Garante que temos dados para comparar
    if df1 is None or df2 is None:
        print("Não é possível comparar: um dos dataframes está vazio.")
        return None, None

    names_df1 = df1['NOME'].dropna().unique()
    names_df2 = df2['NOME'].dropna().unique()

    # 1. Nomes que existiam e SUMIRAM da planilha (Deletados)
    missing_in_df2 = []
    for name in names_df1:
        if name not in names_df2:
            person_data = df1[df1['NOME'] == name].iloc[0]
            missing_in_df2.append({
                'NOME': name,
                'CPF': person_data.get('CPF', 'N/A'),
                'NASCIMENTO': person_data.get('NASCIMENTO', 'N/A')
            })

    # 2. Nomes que não existiam e APARECERSAM (Novos cadastros)
    missing_in_df1 = []
    for name in names_df2:
        if name not in names_df1:
            person_data = df2[df2['NOME'] == name].iloc[0]
            missing_in_df1.append({
                'NOME': name,
                'CPF': person_data.get('CPF', 'N/A'),
                'NASCIMENTO': person_data.get('NASCIMENTO', 'N/A')
            })

    # 3. Nomes presentes em ambos, mas mudou status da MENSALIDADE
    entraram_status = []
    sairam_status = []
    
    common_names = list(set(names_df1) & set(names_df2))

    for name in common_names:
        # Pega a primeira ocorrência do nome
        mensalidade_df1 = df1[df1['NOME'] == name]['MENSALIDADE'].iloc[0]
        mensalidade_df2 = df2[df2['NOME'] == name]['MENSALIDADE'].iloc[0]

        # Era NaN (sem plano) e agora tem valor -> ENTROU (Ativou)
        if pd.isna(mensalidade_df1) and not pd.isna(mensalidade_df2):
            person_data = df2[df2['NOME'] == name].iloc[0]
            entraram_status.append({
                'NOME': name,
                'CPF': person_data.get('CPF', 'N/A'),
                'NASCIMENTO': person_data.get('NASCIMENTO', 'N/A')
            })
        
        # Tinha valor e agora é NaN (sem plano) -> SAIU (Cancelou)
        elif not pd.isna(mensalidade_df1) and pd.isna(mensalidade_df2):
            person_data = df1[df1['NOME'] == name].iloc[0]
            sairam_status.append({
                'NOME': name,
                'CPF': person_data.get('CPF', 'N/A'),
                'NASCIMENTO': person_data.get('NASCIMENTO', 'N/A')
            })

    # Consolidando listas
    lista_entrada = missing_in_df1 + entraram_status
    lista_saida = missing_in_df2 + sairam_status

    # Imprimindo os resultados
    print("-" * 60)
    print(f"RELATÓRIO COMPARATIVO: {month1_name} vs {month2_name}")
    print("-" * 60)

    print(f"🟢 PESSOAS QUE ENTRARAM (Novos ou Reativados): {len(lista_entrada)}")
    if lista_entrada:
        for item in lista_entrada:
             print(f"   + {item['NOME']} | CPF: {item['CPF']}")
    else:
        print("   (Ninguém)")
    print("\n")

    print(f"🔴 PESSOAS QUE SAÍRAM (Cancelados ou Removidos): {len(lista_saida)}")
    if lista_saida:
        for item in lista_saida:
             print(f"   - {item['NOME']} | CPF: {item['CPF']}")
    else:
        print("   (Ninguém)")
    print("-" * 60)

    # Gerar dataframes se solicitado
    df_entrada = pd.DataFrame(lista_entrada) if gerar_df_entrada else None
    df_saida = pd.DataFrame(lista_saida) if gerar_df_saida else None

    if gerar_df_entrada and df_entrada is not None and not df_entrada.empty:
        df_entrada.to_excel('../Data/relatorio_entradas.xlsx', index=False)
        print('📁 Arquivo "relatorio_entradas.xlsx" na pasta "Data".')
    
    if gerar_df_saida and df_saida is not None and not df_saida.empty:
        df_saida.to_excel('../Data/relatorio_saidas.xlsx', index=False)
        print('📁 Arquivo "relatorio_saidas.xlsx" na pasta "Data".')
    
    return df_entrada, df_saida

### Executar Comparação

In [ ]:
# Executa a função
if 'dados_anterior' in locals() and dados_anterior is not None:
    compare_months(
        df1=dados_anterior, 
        df2=dados_atual, 
        month1_name=nome_mes_anterior, 
        month2_name=nome_mes_atual,
        gerar_df_entrada=True,
        gerar_df_saida=True
    )
else:
    print("Não foi possível executar a comparação. Verifique o carregamento dos dados.")